# Flattening the curve in Brazil 
> Confirmed cases that are estimated to need ICUs and estimated number of available ICUs for covid-19 patients

- toc: true
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [jupyter]
- image: images/brazil-flatten-the-curve.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from altair_saver import save
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
url_icus = ('https://covid-insumos.saude.gov.br/paineis/insumos/lista_csv_painel.php?output=csv')
src_icus = pd.read_csv(url_icus, sep=';')

In [3]:
#hide
STATE_COLUMN = "State"
ICUS_COLUMN = 'ICUs'
src_icus = src_icus.rename(columns={'Leitos UTI adulto':ICUS_COLUMN, 'uf':STATE_COLUMN})

In [4]:
#hide
TOTAL_ICU = src_icus[ICUS_COLUMN].sum()

In [5]:
#hide
DATE_COLUMN = "Date"
CONFIRMED_CASES = 'Confirmed Cases'
CASES_NEED_ICU = 0.05
ICU_PER_100k = 20
AVAILABLE_PERCENTAGE_ICU = 0.35
TOTAL_AVAILABLE_ICU = TOTAL_ICU * AVAILABLE_PERCENTAGE_ICU
CASES_NEED_ICU_COLUMN = "Need ICU "+"{:.0f}".format(CASES_NEED_ICU*100)+"% of cases"
PATIENTS = "Patients"
UPPER_BOUND_ICU_NEED_STR = "Upper bound ICU need"
LOWER_BOUND_ICU_NEED_STR = "Lower bound ICU need"
UPPER_BOUND_ICU_NEED = 0.12
LOWER_BOUND_ICU_NEED = 0.01

In [6]:
#hide
url = ('https://data.brasil.io/dataset/covid19/caso.csv.gz')
src = pd.read_csv(url)

In [7]:
#hide
df = src[(src['place_type'] == 'state')]
df = df.rename(columns={
     "state": STATE_COLUMN, 
     "date":DATE_COLUMN, 
     "confirmed":CONFIRMED_CASES
})
# hold a backup
backup = df.copy()

In [8]:
#hide
cols = df.columns[~df.columns.isin([DATE_COLUMN, STATE_COLUMN, CONFIRMED_CASES])]
df = df.drop(cols, axis=1)

In [9]:
#hide
# copy latest value on empty last items
df = df.set_index(STATE_COLUMN)
df = df.reset_index()
df = df.sort_values(by=[DATE_COLUMN], ascending=True)
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
LAST_DATE = max(df[DATE_COLUMN])
while True:
    copied_registers = []
    for row in df.groupby(STATE_COLUMN).last().iterrows():
        if row[1][DATE_COLUMN] < LAST_DATE:
            copied_registers.append([row[0], row[1][DATE_COLUMN] + pd.DateOffset(days=1), 
                          row[1][CONFIRMED_CASES]])
    for row in copied_registers:
        print(row)
        df.loc[row[0]] = [row[0],row[1],row[2]]
        df = df.set_index(STATE_COLUMN)
        df = df.reset_index()
    if ((df.groupby(STATE_COLUMN).last()[DATE_COLUMN] == LAST_DATE).all()):
        break
        
df = df.sort_values(by=[DATE_COLUMN], ascending=True)
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')

['AC', Timestamp('2020-05-16 00:00:00'), 1785]
['AM', Timestamp('2020-05-15 00:00:00'), 17181]
['BA', Timestamp('2020-05-16 00:00:00'), 8128]
['DF', Timestamp('2020-05-16 00:00:00'), 3787]
['ES', Timestamp('2020-05-15 00:00:00'), 5813]
['MA', Timestamp('2020-05-16 00:00:00'), 11592]
['PA', Timestamp('2020-05-16 00:00:00'), 12626]
['PR', Timestamp('2020-05-16 00:00:00'), 2167]
['RJ', Timestamp('2020-05-16 00:00:00'), 19987]
['RN', Timestamp('2020-05-15 00:00:00'), 2786]
['RO', Timestamp('2020-05-16 00:00:00'), 1794]
['RR', Timestamp('2020-05-16 00:00:00'), 1589]
['SC', Timestamp('2020-05-16 00:00:00'), 4562]
['SE', Timestamp('2020-05-16 00:00:00'), 2868]
['TO', Timestamp('2020-05-16 00:00:00'), 1279]
['AM', Timestamp('2020-05-16 00:00:00'), 17181]
['ES', Timestamp('2020-05-16 00:00:00'), 5813]
['RN', Timestamp('2020-05-16 00:00:00'), 2786]


In [10]:
#hide
# get total per day and per state
df = df[[STATE_COLUMN,DATE_COLUMN,CONFIRMED_CASES]].groupby([STATE_COLUMN,DATE_COLUMN],as_index = False).sum().pivot(STATE_COLUMN,DATE_COLUMN).fillna(0)
total = df.sum(axis=0)
df.loc['Total',CONFIRMED_CASES]=total
df = df.reset_index()

In [11]:
#hide
# unpivot data
df = df.melt(id_vars=[STATE_COLUMN], value_vars=[CONFIRMED_CASES])
df = df.drop([None], axis=1)
df = df.rename(columns={'value':CONFIRMED_CASES})

In [12]:
#hide
# add lower and upper bounds
df = df[(df[STATE_COLUMN] == 'Total')]
df[UPPER_BOUND_ICU_NEED_STR] = round(df[CONFIRMED_CASES] * (UPPER_BOUND_ICU_NEED),0)
df[LOWER_BOUND_ICU_NEED_STR] = round(df[CONFIRMED_CASES] * (LOWER_BOUND_ICU_NEED),0)
df[CONFIRMED_CASES] = round(df[CONFIRMED_CASES] * (CASES_NEED_ICU),0)

In [13]:
#hide_input
selection = alt.selection_single(fields=[STATE_COLUMN], name=' ')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_line().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(CONFIRMED_CASES+':Q', axis=alt.Axis(
        title=["Estimated of confirmed cases that need ICU: {:.0f}".format(CASES_NEED_ICU*100)+"% (varies between {:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"% and {:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"%)"])),
    color=color,
    tooltip=[
        DATE_COLUMN, 
        STATE_COLUMN, 
        CONFIRMED_CASES
    ],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Flatten the curve - Brazil",
        "Confirmed cases that are estimated to need ICU (see assumptions)"
    ]
).add_selection(
    selection
)

shades = alt.Chart(df).mark_area().encode(
    x=DATE_COLUMN+':O',
    y=LOWER_BOUND_ICU_NEED_STR+":Q",
    y2=UPPER_BOUND_ICU_NEED_STR+':Q',
    opacity = alt.condition(selection, alt.value(0.2), alt.value(0.5))
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=-10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("ICUs for COVID19 patients: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU)+" units")
)
text2 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% of "+"{:.0f}".format(TOTAL_ICU)+" ICUs in the country")   
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + text2 + shades
plot

alt.LayerChart(...)

In [14]:
#hide
# get only states
df = backup[(backup[STATE_COLUMN] != 'Total')]
cols = df.columns[~df.columns.isin([DATE_COLUMN, STATE_COLUMN, CONFIRMED_CASES])]
df = df.drop(cols, axis=1)
#df.tail()

In [15]:
#hide
# add lower and upper bounds
df[CASES_NEED_ICU_COLUMN] = round(df[CONFIRMED_CASES] * (CASES_NEED_ICU),0)
df[LOWER_BOUND_ICU_NEED_STR] = round(df[CONFIRMED_CASES] * (LOWER_BOUND_ICU_NEED),0)
df[UPPER_BOUND_ICU_NEED_STR] = round(df[CONFIRMED_CASES] * (UPPER_BOUND_ICU_NEED),0)
src_icus = src_icus.set_index(STATE_COLUMN)
df = df.set_index(STATE_COLUMN)
df[ICUS_COLUMN] = src_icus[ICUS_COLUMN].astype(float)
src_icus = src_icus.reset_index()
df = df.reset_index()

In [16]:
#hide
# copy latest value on empty last items
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df = df.sort_values(by=[DATE_COLUMN])
LAST_DATE = max(df[DATE_COLUMN])
while ((df.groupby(STATE_COLUMN).last()[DATE_COLUMN] < LAST_DATE).any()):
    copied_registers = []
    for row in df.groupby(STATE_COLUMN).last().iterrows():
        if row[1][DATE_COLUMN] < LAST_DATE:
            copied_registers.append([row[0], row[1][DATE_COLUMN] + pd.DateOffset(days=1), 
                          row[1][CONFIRMED_CASES],
                          row[1][CASES_NEED_ICU_COLUMN],
                          row[1][LOWER_BOUND_ICU_NEED_STR],
                          row[1][UPPER_BOUND_ICU_NEED_STR],
                          row[1][ICUS_COLUMN]])
    for row in copied_registers:
        df.loc[row[0]] = [row[0],row[1],row[2],row[3],row[4],row[5],row[6]]
        df = df.set_index(STATE_COLUMN)
        df = df.reset_index()
df = df.sort_values(by=[DATE_COLUMN], ascending=True)
df[DATE_COLUMN] = df[DATE_COLUMN].dt.strftime('%m/%d/%y')

In [17]:
#hide_input
input_dropdown = alt.binding_select(options=df[STATE_COLUMN].unique())
selection = alt.selection_single(
    fields=[STATE_COLUMN], 
    init={STATE_COLUMN:'SC'}, 
    bind=input_dropdown, 
    name=' ')
color = alt.condition(selection,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='Paired', reverse=True), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_line().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(CASES_NEED_ICU_COLUMN+':Q', axis=alt.Axis(
        title=["Estimated of confirmed cases that need ICU: {:.0f}".format(CASES_NEED_ICU*100)+"% (varies between {:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"% and {:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"%)"])),
    color=color,
    tooltip=[
        DATE_COLUMN, 
        STATE_COLUMN, 
        CASES_NEED_ICU_COLUMN
    ],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Flatten the curve per state",
        "Confirmed cases that are estimated to need ICU (see assumptions)"
    ]
).add_selection(
    selection
).transform_filter(
    selection
)

shades = alt.Chart(df).mark_area().encode(
    x=DATE_COLUMN+':O',
    y=LOWER_BOUND_ICU_NEED_STR+":Q",
    y2=UPPER_BOUND_ICU_NEED_STR+':Q',
    color=color,
    opacity = alt.condition(selection, alt.value(0.2), alt.value(0.5))
).transform_filter(
    selection
)

x1line = alt.Chart(src_icus).mark_rule(color='#e42726', strokeWidth=2).encode(
    y=ICUS_COLUMN+':Q'
).transform_filter(
    selection
)

text1 = alt.Chart(src_icus).mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    y=ICUS_COLUMN+':Q',
    text=alt.value("{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% of ICU beds")   
).transform_filter(
    selection
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + shades
plot

alt.LayerChart(...)

In [18]:
#hide_input
df.groupby(STATE_COLUMN).last()

,Date,Confirmed Cases,Need ICU 5% of cases,Lower bound ICU need,Upper bound ICU need,ICUs
State,,,,,,
AC,05/16/20,1785,89.0,18.0,214.0,58.0
AL,05/16/20,3593,180.0,36.0,431.0,426.0
AM,05/16/20,17181,859.0,172.0,2062.0,450.0
AP,05/16/20,3834,192.0,38.0,460.0,78.0
BA,05/16/20,8128,406.0,81.0,975.0,1573.0
CE,05/16/20,23795,1190.0,238.0,2855.0,1002.0
DF,05/16/20,3787,189.0,38.0,454.0,997.0
ES,05/16/20,5813,291.0,58.0,698.0,804.0
GO,05/16/20,1652,83.0,17.0,198.0,1200.0


In [19]:
#hide_input
print("Assumptions:")
print("- Considering ICUs for adults, both public and private (source: painel de insumos e leitos).")
print("- {:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% of the ICUs would be available for covid-19 patients (source: oglobo)")
print("- It is estimated that "+"{:.0f}".format(CASES_NEED_ICU*100)+"% of covid-19 patients will need ICU. In Italy it reached 12% and in China, the lower bound, it was around 1% of the confirmed cases (source: the lancet paper)")

Assumptions:
- Considering ICUs for adults, both public and private (source: painel de insumos e leitos).
- 35% of the ICUs would be available for covid-19 patients (source: oglobo)
- It is estimated that 5% of covid-19 patients will need ICU. In Italy it reached 12% and in China, the lower bound, it was around 1% of the confirmed cases (source: the lancet paper)


In [20]:
#hide_input
HTML(f'<small class="float-right">Last update on {pd.to_datetime(LAST_DATE).strftime("%d/%m/%Y")}</small>')

Developed by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). Based on the work of [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html). 

Data sources:
- covid19 data: [CSSEGISandData](https://github.com/CSSEGISandData/COVID-19)
- number of ICUs: [painel de insumos e leitos](https://covid-insumos.saude.gov.br/paineis/insumos/painel_leitos.php), [elpais](https://brasil.elpais.com/brasil/2020-04-28/sem-transparencia-sobre-fila-para-utis-justica-opera-para-garantir-atendimento-a-pacientes-de-covid-19.html), [elpais2](https://brasil.elpais.com/sociedade/2020-04-15/sem-leitos-de-uti-municipios-pequenos-temem-por-estrutura-limitada-para-transferir-pacientes-graves-com-a-covid-19.html), [bcc](https://www.bbc.com/portuguese/brasil-52137553) and [ministério da saúde](https://www.saude.gov.br/noticias/agencia-saude/46772-brasil-ganha-reforco-de-1-134-leitos-de-uti-no-combate-ao-coronavirus)
- available ICUs: [oglobo](https://oglobo.globo.com/sociedade/coronavirus/coronavirus-ministerio-estima-que-sus-tem-de-12-13-mil-leitos-de-uti-disponiveis-para-atender-pacientes-1-24328523)
- patients that need ICU: [the lancet paper](https://linkinghub.elsevier.com/retrieve/pii/S2213260020301612)

In [21]:
#hide
save(plot,"../images/brazil-flatten-the-curve.png")

ValueError: No enabled saver found that supports format='png'